In [37]:
import pandas as pd
import os

In [38]:
# Step up to project root from /notebooks/
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Use project_root to build paths
raw_data_path = os.path.join(project_root, "data", "raw")
processed_data_path = os.path.join(project_root, "data", "processed")

food_file = os.path.join(raw_data_path, "Food_Access_Raw.csv")
disease_file = os.path.join(raw_data_path, "Chronic_Diseases_Raw.csv")
output_file = os.path.join(processed_data_path, "merged_summary.csv")

In [39]:
# Load datasets
food_df = pd.read_csv(food_file, low_memory=False)
disease_df = pd.read_csv(disease_file, low_memory=False)

### Food Access Summary

In [44]:
# Group by state and calculate percent of tracts marked as LILATracts_1And10
food_summary = (
    food_df.groupby("State")
    .agg(
        total_tracts=("CensusTract", "count"),
        lilatracts_1and10=("LILATracts_1And10", "sum"),
        lilatracts_halfand10=("LILATracts_halfAnd10", "sum"),
        poverty_rate_avg=("PovertyRate", "mean"),
        median_income_avg=("MedianFamilyIncome", "mean")
    )
    .assign(
        percent_lila_1and10=lambda df: 100 * df.lilatracts_1and10 / df.total_tracts,
        percent_lila_halfand10=lambda df: 100 * df.lilatracts_halfand10 / df.total_tracts
    )
    .reset_index()
)

### Chronic Disease Filtering

In [50]:
# Standardize disease terms
relevant_topics = ["Nutrition, Physical Activity, and Weight Status", "Diabetes", "Cardiovascular Disease"]
relevant_responses = ["Crude Prevalence", "Age-adjusted Prevalence", "Proportion"]

disease_filtered = disease_df[
    (disease_df["Topic"].isin(relevant_topics)) &
    (disease_df["DataValueType"].isin(relevant_responses)) &
    (disease_df["StratificationCategory1"] == "Overall")
]

# Keep only the latest year per state/topic/question
disease_latest = (
    disease_filtered.sort_values("YearStart", ascending=False)
    .drop_duplicates(subset=["State", "Topic", "Question"])
)

disease_latest = disease_latest.drop(columns=["YearStart", "YearEnd"])

disease_pivot = disease_latest.pivot_table(
    index="State",
    columns="Question",
    values="DataValue",
    aggfunc="first"
).reset_index()


### Merge

In [52]:
merged = pd.merge(food_summary, disease_pivot, on="State", how="inner")

In [53]:
# Save result
os.makedirs(processed_data_path, exist_ok=True)
merged.to_csv(output_file, index=False)

print(f"✅ Processed data saved to {output_file}")

✅ Processed data saved to C:\Users\mjben\PycharmProjects\Food_Deserts_Capstone\data\processed\merged_summary.csv
